In [1]:
!pip install pysmiles

  Using cached pbr-5.8.0-py2.py3-none-any.whl (112 kB)
  Created wheel for pysmiles: filename=pysmiles-1.0.1-py2.py3-none-any.whl size=22028 sha256=6345f5a1e8593856b044f3358131ddf4bc9adb249f7a3b215b2556169544407b
  Stored in directory: /root/.cache/pip/wheels/e4/f0/ca/dae2e932684a6e26824d29cf5b6dadea7320e6fed036942972
Successfully built pysmiles


In [113]:
import pandas as pd
from pysmiles import read_smiles
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from random import randint
from math import sqrt

In [3]:
smiles_dfwH = pd.read_csv("OriginalSMILES_QM9.csv")
smiles_dfwH = smiles_dfwH.drop('idx', axis=1)

In [4]:
smiles_dfwH

,smiles
0,C
1,N
2,O
3,C#C
4,C#N
...,...
132035,C1C2C3C4C5OC14C5N23
132036,C1N2C3C2C2C4OC12C34
132037,C1N2C3C4C5C2C13CN45
132038,C1N2C3C4C5CC13C2C45


In [5]:
molmatrixwH = []

for i in range(len(smiles_dfwH)):
  molwH = read_smiles(smiles_dfwH['smiles'][i], explicit_hydrogen=True)
  molmatrixwH.append(nx.to_numpy_matrix(molwH))

In [6]:
countwH = 0

for i in range(len(smiles_dfwH)):
  if len(molmatrixwH[i]) <= 11:
      countwH += 1

In [7]:
lenwH = []

for i in range(len(smiles_dfwH)):
  lenwH.append(len(molmatrixwH[i]))

In [8]:
smiles_dfwH['matrixwH'] = molmatrixwH
smiles_dfwH['lenwH'] = lenwH

In [9]:
smiles_dfwH = smiles_dfwH.drop(smiles_dfwH[smiles_dfwH['lenwH'] > 11].index)

In [10]:
smiles_dfwH = smiles_dfwH.reset_index()
smiles_dfwH = smiles_dfwH.drop('index', axis=1)

In [11]:
smiles_dfwH

,smiles,matrixwH,lenwH
0,C,"[[[[[0. 1. 1. 1. 1.]]]], [[[[1. 0. 0. 0. 0.]]]...",5
1,N,"[[[[[0. 1. 1. 1.]]]], [[[[1. 0. 0. 0.]]]], [[[...",4
2,O,"[[[[[0. 1. 1.]]]], [[[[1. 0. 0.]]]], [[[[1. 0....",3
3,C#C,"[[[[[0. 1. 1. 0.]]]], [[[[1. 0. 0. 1.]]]], [[[...",4
4,C#N,"[[[[[0. 1. 1.]]]], [[[[1. 0. 0.]]]], [[[[1. 0....",3
...,...,...,...
1939,FC(F)(F)C1(CO1)C#N,"[[[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]], [[[...",11
1940,FC(F)(F)C1OC1C#N,"[[[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]], [[[...",11
1941,FC(F)(F)CC#CC#N,"[[[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]], [[[...",11
1942,FC(F)(F)CC(=O)C#N,"[[[[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]], [[[...",11


In [12]:
smiles_csv_data = smiles_dfwH.to_csv('Graph_QM9_Hydrogens.csv', index = True)

In [13]:
def print_adj_list(adj):
    for i in range(len(adj)):
        print(str(i) + " -> ", end='')
        for x in range(len(adj[i])-1):
            print(adj[i][x], end=', ')
        print(adj[i][-1])

In [14]:
def print_adj_matrix(matrix):
    for i in range(len(matrix)):
        for j in range(len(matrix[i])-1):
            print(matrix[i][j], end=' ')
        print(matrix[i][-1])

In [15]:
def pad_matrix(matrix, buf=11):
    for i in range(len(matrix)):
        for j in range(buf - len(matrix[i])):
            matrix[i].append(-1)
    for i in range(buf-len(matrix)):
        matrix.append([-1] * buf)
    return matrix

In [121]:
def adj_matrix_to_list(matrix, buf=11):
    matrix = pad_matrix(matrix)
    try: adj = np.reshape(matrix, (1, buf**2))
    except ValueError: return None
    else: return adj

In [21]:
graph_hydrogens = pd.read_csv("Graph_QM9_Hydrogens.csv").drop('smiles', axis=1).drop('lenwH', axis=1)

In [56]:
graph_hydrogens

,Unnamed: 0,matrixwH
0,0,[[0. 1. 1. 1. 1.]\n [1. 0. 0. 0. 0.]\n [1. 0. ...
1,1,[[0. 1. 1. 1.]\n [1. 0. 0. 0.]\n [1. 0. 0. 0.]...
2,2,[[0. 1. 1.]\n [1. 0. 0.]\n [1. 0. 0.]]
3,3,[[0. 1. 1. 0.]\n [1. 0. 0. 1.]\n [1. 0. 0. 0.]...
4,4,[[0. 1. 1.]\n [1. 0. 0.]\n [1. 0. 0.]]
...,...,...
1939,1939,[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [1. 0. 1...
1940,1940,[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [1. 0. 1...
1941,1941,[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [1. 0. 1...
1942,1942,[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [1. 0. 1...


In [105]:
def str_to_adj_matrix(tmp):
    tmp = tmp.replace("\n", ",").replace(".", "")
    tmp = list(tmp)
    tmp = list(filter(lambda x: x not in ['[', ']', ' ', ','], tmp))
    out = []
    idx = 0
    for i in range(int(sqrt(len(tmp)))):
        x = []
        for j in range(int(sqrt(len(tmp)))):
            x.append(int(tmp[idx]))
            idx += 1
        out.append(x)
    return out

In [124]:
import sys
sys.stdout = open("converted_mols.txt", "w")

In [125]:
for i in range(len(graph_hydrogens)):
    print(adj_matrix_to_list(str_to_adj_matrix(graph_hydrogens['matrixwH'][i])))